# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 21 2023 1:07PM,257855,10,Yusen-9188065,Yusen – 3D Matrix,Released
1,Feb 21 2023 1:07PM,257855,10,Yusen-9188066,Yusen – 3D Matrix,Released
2,Feb 21 2023 1:07PM,257855,10,Yusen-9188073,Yusen – 3D Matrix,Released
3,Feb 21 2023 12:53PM,257853,22,641969,"NBTY Global, Inc.",Released
4,Feb 21 2023 12:53PM,257852,22,641965,"NBTY Global, Inc.",Released
5,Feb 21 2023 12:51PM,257851,19,ACG-2102492828,ACG North America LLC,Released
6,Feb 21 2023 12:34PM,257850,20,9188061,"Exact-Rx, Inc.",Released
7,Feb 21 2023 12:32PM,257848,16,9187518,Sartorius Bioprocess Solutions,Executing
8,Feb 21 2023 12:29PM,257846,10,SO94547,"Senseonics, Incorporated",Released
9,Feb 21 2023 12:28PM,257844,10,SONSB0002112,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,257850,Released,1
51,257851,Released,1
52,257852,Released,1
53,257853,Released,1
54,257855,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257850,NaN,NaN,1.0
257851,NaN,NaN,1.0
257852,NaN,NaN,1.0
257853,NaN,NaN,1.0
257855,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257735,0.0,20.0,2.0
257739,0.0,14.0,5.0
257741,0.0,0.0,1.0
257781,0.0,4.0,9.0
257782,0.0,0.0,58.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257735,0,20,2
257739,0,14,5
257741,0,0,1
257781,0,4,9
257782,0,0,58


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257735,0,20,2
1,257739,0,14,5
2,257741,0,0,1
3,257781,0,4,9
4,257782,0,0,58


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257735,,20,2
1,257739,,14,5
2,257741,,,1
3,257781,,4,9
4,257782,,,58


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 21 2023 1:07PM,257855,10,Yusen – 3D Matrix
3,Feb 21 2023 12:53PM,257853,22,"NBTY Global, Inc."
4,Feb 21 2023 12:53PM,257852,22,"NBTY Global, Inc."
5,Feb 21 2023 12:51PM,257851,19,ACG North America LLC
6,Feb 21 2023 12:34PM,257850,20,"Exact-Rx, Inc."
7,Feb 21 2023 12:32PM,257848,16,Sartorius Bioprocess Solutions
8,Feb 21 2023 12:29PM,257846,10,"Senseonics, Incorporated"
9,Feb 21 2023 12:28PM,257844,10,"Nextsource Biotechnology, LLC"
10,Feb 21 2023 12:28PM,257845,10,ISDIN Corporation
12,Feb 21 2023 12:20PM,257841,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 21 2023 1:07PM,257855,10,Yusen – 3D Matrix,,,3
1,Feb 21 2023 12:53PM,257853,22,"NBTY Global, Inc.",,,1
2,Feb 21 2023 12:53PM,257852,22,"NBTY Global, Inc.",,,1
3,Feb 21 2023 12:51PM,257851,19,ACG North America LLC,,,1
4,Feb 21 2023 12:34PM,257850,20,"Exact-Rx, Inc.",,,1
5,Feb 21 2023 12:32PM,257848,16,Sartorius Bioprocess Solutions,,1,
6,Feb 21 2023 12:29PM,257846,10,"Senseonics, Incorporated",,,1
7,Feb 21 2023 12:28PM,257844,10,"Nextsource Biotechnology, LLC",,,1
8,Feb 21 2023 12:28PM,257845,10,ISDIN Corporation,,,2
9,Feb 21 2023 12:20PM,257841,10,"Methapharm, Inc.",,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 1:07PM,257855,10,Yusen – 3D Matrix,3,,
1,Feb 21 2023 12:53PM,257853,22,"NBTY Global, Inc.",1,,
2,Feb 21 2023 12:53PM,257852,22,"NBTY Global, Inc.",1,,
3,Feb 21 2023 12:51PM,257851,19,ACG North America LLC,1,,
4,Feb 21 2023 12:34PM,257850,20,"Exact-Rx, Inc.",1,,
5,Feb 21 2023 12:32PM,257848,16,Sartorius Bioprocess Solutions,,1,
6,Feb 21 2023 12:29PM,257846,10,"Senseonics, Incorporated",1,,
7,Feb 21 2023 12:28PM,257844,10,"Nextsource Biotechnology, LLC",1,,
8,Feb 21 2023 12:28PM,257845,10,ISDIN Corporation,2,,
9,Feb 21 2023 12:20PM,257841,10,"Methapharm, Inc.",29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 1:07PM,257855,10,Yusen – 3D Matrix,3,,
1,Feb 21 2023 12:53PM,257853,22,"NBTY Global, Inc.",1,,
2,Feb 21 2023 12:53PM,257852,22,"NBTY Global, Inc.",1,,
3,Feb 21 2023 12:51PM,257851,19,ACG North America LLC,1,,
4,Feb 21 2023 12:34PM,257850,20,"Exact-Rx, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 1:07PM,257855,10,Yusen – 3D Matrix,3.0,NaN,NaN
1,Feb 21 2023 12:53PM,257853,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Feb 21 2023 12:53PM,257852,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Feb 21 2023 12:51PM,257851,19,ACG North America LLC,1.0,NaN,NaN
4,Feb 21 2023 12:34PM,257850,20,"Exact-Rx, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 1:07PM,257855,10,Yusen – 3D Matrix,3.0,0.0,0.0
1,Feb 21 2023 12:53PM,257853,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Feb 21 2023 12:53PM,257852,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Feb 21 2023 12:51PM,257851,19,ACG North America LLC,1.0,0.0,0.0
4,Feb 21 2023 12:34PM,257850,20,"Exact-Rx, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8250252,398.0,3.0,0.0
12,257810,1.0,0.0,0.0
15,1288785,28.0,53.0,9.0
16,257848,0.0,1.0,0.0
19,773475,4.0,21.0,10.0
20,257850,1.0,0.0,0.0
21,515648,2.0,0.0,0.0
22,515705,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8250252,398.0,3.0,0.0
1,12,257810,1.0,0.0,0.0
2,15,1288785,28.0,53.0,9.0
3,16,257848,0.0,1.0,0.0
4,19,773475,4.0,21.0,10.0
5,20,257850,1.0,0.0,0.0
6,21,515648,2.0,0.0,0.0
7,22,515705,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,398.0,3.0,0.0
1,12,1.0,0.0,0.0
2,15,28.0,53.0,9.0
3,16,0.0,1.0,0.0
4,19,4.0,21.0,10.0
5,20,1.0,0.0,0.0
6,21,2.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,398.0
1,12,Released,1.0
2,15,Released,28.0
3,16,Released,0.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,9.0,0.0,10.0,0.0,0.0,0.0
Executing,3.0,0.0,53.0,1.0,21.0,0.0,0.0,0.0
Released,398.0,1.0,28.0,0.0,4.0,1.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,9.0,0.0,10.0,0.0,0.0,0.0
1,Executing,3.0,0.0,53.0,1.0,21.0,0.0,0.0,0.0
2,Released,398.0,1.0,28.0,0.0,4.0,1.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,9.0,0.0,10.0,0.0,0.0,0.0
1,Executing,3.0,0.0,53.0,1.0,21.0,0.0,0.0,0.0
2,Released,398.0,1.0,28.0,0.0,4.0,1.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()